# Reading CSV Data 

In [1]:
import pandas as pd
import numpy as np
import numpy as np
import matplotlib.pyplot as plt

incidents = pd.read_csv('Traffic_Incidents.csv')
incidents['START_DT'] = pd.to_datetime(incidents['START_DT'])
incidents = incidents.loc[incidents['START_DT'].dt.year == 2018, :]
incidents = incidents.rename(columns = {'Latitude' : 'latitude', 'Longitude':'longitude'})
incidents = incidents[[ "latitude","longitude",]]
incidents['longitude'].sort_values()
incidents['latitude'].sort_values()

signals = pd.read_csv('Traffic_Signals.csv')
signals = signals[["latitude", 'longitude']]
signals

cameras = pd.read_csv('Traffic_Camera_Locations.csv')
cameras = cameras[['latitude', 'longitude']]
cameras

speed = pd.read_csv('Speed_Limits.csv')
speed['multiline'] = speed['multiline'].map(lambda x: x.lstrip('MULTILINESTRING (('))
speed['multiline'] = speed['multiline'].str.split(',').str[0]
speed['longitude'] = speed['multiline'].str.split(' ').str[0]
speed['latitude'] = speed['multiline'].str.split(' ').str[1]
speed = speed[['latitude', 'longitude', 'SPEED']]
speed = speed.apply(pd.to_numeric)

volume = pd.read_csv('Traffic_Volumes_for_2018.csv')
volume['multilinestring'] = volume['multilinestring'].map(lambda x: x.lstrip('MULTILINESTRING (('))
volume['multilinestring'] = volume['multilinestring'].str.split(',').str[0]
volume['longitude'] = volume['multilinestring'].str.split(' ').str[0]
volume['latitude'] = volume['multilinestring'].str.split(' ').str[1]
volume = volume[['VOLUME', 'latitude', 'longitude']]
volume = volume.apply(pd.to_numeric)

signs = pd.read_csv('Traffic_Signs.csv')
signs['POINT'] = signs['POINT'].map(lambda x: x.lstrip('POINT ('))
signs['POINT'] = signs['POINT'].map(lambda x: x.rstrip(')'))
signs['latitude'] = signs['POINT'].str.split(' ').str[1]
signs['longitude'] = signs['POINT'].str.split( ).str[0]
signs = signs[['latitude','longitude']]
signs = signs.apply(pd.to_numeric)

display(incidents, signals, cameras, speed, volume, signs)


,latitude,longitude
875,51.065699,-114.028193
2169,51.021205,-114.059276
3262,51.099956,-113.958764
4559,51.065739,-114.028183
4695,50.892629,-114.061310
...,...,...
17407,51.067053,-114.020057
17408,51.052492,-114.118501
17409,51.049133,-113.956571
17410,51.054765,-114.079493


,latitude,longitude
0,51.052514,-114.091740
1,51.154211,-114.052475
2,51.094081,-114.011478
3,51.117992,-114.069588
4,51.037739,-114.025122
...,...,...
1533,51.154147,-114.228956
1534,50.902069,-114.109199
1535,51.073213,-114.084589
1536,51.048341,-114.060387


,latitude,longitude
0,50.900726,-113.976606
1,51.053253,-113.955818
2,51.098849,-114.149379
3,51.111255,-114.178204
4,51.139352,-114.001451
...,...,...
121,51.050802,-114.050136
122,50.979446,-113.981495
123,50.979635,-113.929263
124,51.048677,-114.073644


,latitude,longitude,SPEED
0,50.913577,-114.073658,60
1,50.996224,-114.034764,40
2,51.166714,-114.068566,60
3,51.005002,-114.117825,70
4,51.062448,-113.918851,80
...,...,...,...
1447,51.091535,-114.233242,80
1448,50.917759,-114.140310,70
1449,50.904313,-114.030210,60
1450,51.124175,-114.177871,60


,VOLUME,latitude,longitude
0,22000,51.048319,-114.060367
1,5000,50.968634,-114.068768
2,5000,51.053232,-114.033722
3,2000,51.143370,-114.013188
4,14000,50.957841,-113.981288
...,...,...,...
1764,44000,51.154205,-114.027976
1765,27000,50.972345,-113.981983
1766,20000,51.065390,-114.013382
1767,7000,50.875745,-114.035671


,latitude,longitude
0,51.068567,-114.210957
1,50.911166,-113.935804
2,51.042028,-114.054231
3,51.041351,-114.053597
4,50.911341,-113.935926
...,...,...
207515,50.914135,-114.089567
207516,51.050466,-114.073475
207517,51.052801,-114.090109
207518,50.974158,-114.053874


In [2]:
signs = signs.sort_values(['latitude', 'longitude'], ascending = [1,0])
signs

,latitude,longitude
203219,50.848468,-114.023890
54500,50.848508,-114.024579
61501,50.848551,-114.027426
122253,50.848551,-114.027426
18229,50.848568,-114.029795
...,...,...
43950,51.212619,-114.030909
3616,51.212632,-114.029625
98614,51.212663,-114.071434
178106,51.212671,-114.048203


# Creating our Grid

In [17]:
boundary = pd.read_csv('City_Boundary_layer.csv')

boundary['the_geom'] = boundary['the_geom'].map(lambda x: x.lstrip('POLYGON (('))

boundary.the_geom.apply(pd.Series)

boundarySplit = boundary['the_geom'].str.split(',')

x = boundarySplit[0]

lat = [float(y.split()[1].strip('))')) for y in x]
long = [float(y.split()[0].strip('))')) for y in x]

endlat = max(lat)
startlat = min(lat)
startlong = min(long)
endlong = max(long)

lowerLeft = [startlat, startlong]
upperRight = [endlat, endlong]

def get_geojson_grid(upper_right, lower_left, n, m):
    """Returns a grid of geojson rectangles, and computes the exposure in each section of the grid based on the vessel data.
    Parameters
    ----------
    upper_right: array_like
        The upper right hand corner of "grid of grids" (the default is the upper right hand [lat, lon] of the USA).
    lower_left: array_like
        The lower left hand corner of "grid of grids"  (the default is the lower left hand [lat, lon] of the USA).
    n: integer
        The number of rows/columns in the (n,n) grid.
    list
        List of "geojson style" dictionary objects   
    """
    all_boxes = []

    lat_steps = np.linspace(lower_left[0], upper_right[0], n+1)
    lon_steps = np.linspace(lower_left[1], upper_right[1], m+1)

    lat_stride = lat_steps[1] - lat_steps[0]
    lon_stride = lon_steps[1] - lon_steps[0]

    for lat in lat_steps[:-1]:
        for lon in lon_steps[:-1]:
            # Define dimensions of box in grid
            upper_left = [lon, lat + lat_stride]
            upper_right = [lon + lon_stride, lat + lat_stride]
            lower_right = [lon + lon_stride, lat]
            lower_left = [lon, lat]

        
            geo_json = {
                        "lower_left": lower_left,
                        "lower_right": lower_right,
                            "upper_left": upper_left,
                            "upper_right": upper_right
                        }

          
            all_boxes.append(geo_json)

    return all_boxes

gridLines = get_geojson_grid(upperRight, lowerLeft,10,10)

df = pd.DataFrame(data = gridLines)
df = df.astype(str)

df['minX'] = df["lower_left"].str.split(',').str[1]
df['minX'] = df['minX'].str.split(']').str[0]

df['maxX'] = df['upper_right'].str.split(',').str[1]
df['maxX'] = df['maxX'].str.split(']').str[0]

df['minY'] = df['lower_left'].str.split(',').str[0]
df['minY'] = df['minY'].str.split('[').str[1]

df['maxY'] = df['upper_right'].str.split('[').str[1]
df['maxY'] = df['maxY'].str.split(',').str[0]
             
df = df[['minX','maxX', 'minY', 'maxY']]
df['minX'] = df['minX'].map(lambda x: x.lstrip(' '))
df['maxX'] = df['maxX'].map(lambda x: x.lstrip(' '))
df = df.apply(pd.to_numeric)

#df = df.reindex([55,54,65,45,64,53,63,0,1,2,3,4,5,6,7,8,9,10,11, 12, 13,14, 15, 16, 17, 18, 19,20, 21,22, 23, 24, 25, 26 ,27, 28, 29, 30, 31, 32,33,34,35, 36, 37, 38, 39,40, 41, 42, 43, 44, 46, 47, 48, 49, 50, 51, 52,56,57,58,59,60,61,62,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99])
df

,minX,maxX,minY,maxY
0,50.842822,50.879782,-114.315796,-114.270207
1,50.842822,50.879782,-114.270207,-114.224618
2,50.842822,50.879782,-114.224618,-114.179029
3,50.842822,50.879782,-114.179029,-114.133440
4,50.842822,50.879782,-114.133440,-114.087850
...,...,...,...,...
95,51.175465,51.212425,-114.087850,-114.042261
96,51.175465,51.212425,-114.042261,-113.996672
97,51.175465,51.212425,-113.996672,-113.951083
98,51.175465,51.212425,-113.951083,-113.905494


In [8]:
def checkPoint(dataFrame, series, name):
    """checkPoint takes in a list of tuple argument called points and it will pass each x,y coordinate in the list and if the point is located inside the grid specifications, the count increases.
    """
    df = dataFrame
    s = series
    county = []
    count = 0
    for index, row in df.iterrows():
        for indexy, seriesrows in s.iterrows():
            if (seriesrows['latitude'] >= row['minX'] and seriesrows['latitude'] < row['maxX'] and seriesrows['longitude'] >= row['minY'] and seriesrows['longitude'] < row['maxY']):
                count += 1
                s.drop(indexy, inplace = True)
                    
        county.append(count)
        count = 0
    
    df[name] = county
    return df
        

In [9]:
def checkPointAverage(dataFrame, series, name, parameter):
    """checkPoint takes in a list of tuple argument called points and it will pass each x,y 
    coordinate in the list and if the point is located inside the grid specifications, the count increases.
    """
    df = dataFrame
    s = series
    array = []
    county = []
    sum = 0
    average = 0
    count = 0
    for index, row in df.iterrows():
        for indexy, seriesrows in s.iterrows():
            if (seriesrows['latitude'] >= row['minX'] and seriesrows['latitude'] < row['maxX']):
                if(seriesrows['longitude'] >= row['minY'] and seriesrows['longitude'] < row['maxY']):
                    count += 1
                    sum += seriesrows[parameter]
                    s.drop(indexy, inplace = True)
        if count == 0:
            county.append(count)
            array.append(count)
        else:
            array.append(sum/count)            
            county.append(count)
        count = 0
        sum = 0
        average = 0
    df[parameter] = array
    df[name] = county
    return df


In [18]:
df = checkPoint(df,incidents,"accident")
df.insert(0, 'grids', np.arange(1, len(df) + 1))

In [31]:
df.describe()

,grids,minX,maxX,minY,maxY,accident,signals,cameras,SPEED,Average Speed Count,VOLUME,Average Volume Count
count,100.000000,100.000000,100.000000,100.000000,100.000000,100.0,100.0,100.00000,100.000000,100.000000,100.0,100.0
mean,50.500000,51.009143,51.046104,-114.110645,-114.065056,0.0,0.0,1.26000,46.503923,14.510000,0.0,0.0
std,29.011492,0.106695,0.106695,0.131604,0.131604,0.0,0.0,2.80555,30.357026,15.573543,0.0,0.0
min,1.000000,50.842822,50.879782,-114.315796,-114.270207,0.0,0.0,0.00000,0.000000,0.000000,0.0,0.0
25%,25.750000,50.916743,50.953703,-114.224618,-114.179029,0.0,0.0,0.00000,0.000000,0.000000,0.0,0.0
50%,50.500000,51.009143,51.046104,-114.110645,-114.065056,0.0,0.0,0.00000,60.000000,10.000000,0.0,0.0
75%,75.250000,51.101544,51.138504,-113.996672,-113.951083,0.0,0.0,2.00000,65.496795,25.000000,0.0,0.0
max,100.000000,51.175465,51.212425,-113.905494,-113.859905,0.0,0.0,22.00000,110.000000,52.000000,0.0,0.0


In [20]:
df = checkPoint(df,signals,"signals")


In [21]:
df = checkPoint(df,cameras,"cameras")
df

,grids,minX,maxX,minY,maxY,accident,signals,cameras
0,1,50.842822,50.879782,-114.315796,-114.270207,0,0,0
1,2,50.842822,50.879782,-114.270207,-114.224618,0,0,0
2,3,50.842822,50.879782,-114.224618,-114.179029,0,0,0
3,4,50.842822,50.879782,-114.179029,-114.133440,0,0,0
4,5,50.842822,50.879782,-114.133440,-114.087850,0,0,0
...,...,...,...,...,...,...,...,...
95,96,51.175465,51.212425,-114.087850,-114.042261,0,0,0
96,97,51.175465,51.212425,-114.042261,-113.996672,0,0,0
97,98,51.175465,51.212425,-113.996672,-113.951083,0,0,0
98,99,51.175465,51.212425,-113.951083,-113.905494,0,0,0


In [22]:
df = checkPointAverage(df, speed, "Average Speed Count", "SPEED")
df

,grids,minX,maxX,minY,maxY,accident,signals,cameras,SPEED,Average Speed Count
0,1,50.842822,50.879782,-114.315796,-114.270207,0,0,0,0.000000,0
1,2,50.842822,50.879782,-114.270207,-114.224618,0,0,0,0.000000,0
2,3,50.842822,50.879782,-114.224618,-114.179029,0,0,0,0.000000,0
3,4,50.842822,50.879782,-114.179029,-114.133440,0,0,0,0.000000,0
4,5,50.842822,50.879782,-114.133440,-114.087850,0,0,0,0.000000,0
...,...,...,...,...,...,...,...,...,...,...
95,96,51.175465,51.212425,-114.087850,-114.042261,0,0,0,70.000000,2
96,97,51.175465,51.212425,-114.042261,-113.996672,0,0,0,73.636364,11
97,98,51.175465,51.212425,-113.996672,-113.951083,0,0,0,65.000000,12
98,99,51.175465,51.212425,-113.951083,-113.905494,0,0,0,0.000000,0


In [24]:
df = checkPointAverage(df, volume, "Average Volume Count", "VOLUME")


In [32]:
df[20:56]

,grids,minX,maxX,minY,maxY,accident,signals,cameras,SPEED,Average Speed Count,VOLUME,Average Volume Count
20,21,50.916743,50.953703,-114.315796,-114.270207,0,0,0,0.000000,0,0,0
21,22,50.916743,50.953703,-114.270207,-114.224618,0,0,0,0.000000,0,0,0
22,23,50.916743,50.953703,-114.224618,-114.179029,0,0,0,0.000000,0,0,0
23,24,50.916743,50.953703,-114.179029,-114.133440,0,0,0,59.000000,10,0,0
24,25,50.916743,50.953703,-114.133440,-114.087850,0,0,0,58.235294,17,0,0
25,26,50.916743,50.953703,-114.087850,-114.042261,0,0,5,49.200000,25,0,0
26,27,50.916743,50.953703,-114.042261,-113.996672,0,0,0,59.047619,21,0,0
27,28,50.916743,50.953703,-113.996672,-113.951083,0,0,5,65.384615,26,0,0
28,29,50.916743,50.953703,-113.951083,-113.905494,0,0,0,64.615385,26,0,0
29,30,50.916743,50.953703,-113.905494,-113.859905,0,0,0,61.428571,7,0,0


In [25]:
#Checking if all the data points were used
display(volume,incidents, cameras, signals,speed )

,VOLUME,latitude,longitude


,latitude,longitude
10719,0.00511,-114.039384
16292,0.00511,-114.039384


,latitude,longitude


,latitude,longitude


,latitude,longitude,SPEED
641,50.892359,-113.859821,80


In [7]:
df['accident'].sort_values()

0       0
22      0
29      0
30      0
31      0
     ... 
45    307
54    326
66    354
56    362
55    465
Name: accident, Length: 100, dtype: int64

In [ ]:
from IPython.display import clear_output
import timeit
start = timeit.default_timer()
def checkPointSigns(dataFrame, series, name):
    """checkPoint takes in a list of tuple argument called points and it will pass each x,y 
    coordinate in the list and if the point is located inside the grid specifications, the count increases.
    """
    df = dataFrame
    s = series
    county = []
    count = 0
    for index, row in df.iterrows():
        clear_output(wait=True)
        for indexy, seriesrows in s.iterrows():
            if (seriesrows['latitude'] >= row['minX'] and seriesrows['latitude'] < row['maxX'] and seriesrows['longitude'] >= row['minY'] and seriesrows['longitude'] < row['maxY']):
                count += 1
                s.drop(indexy, inplace = True)
                   
        county.append(count)
        count = 0
        
        
        stop = timeit.default_timer()
        if(index/len(df)*100) < 5:
            expected_time = "Calculating"
        else:
            
            time_perc = timeit.default_timer()
            expected_time = np.round ( ( (time_perc - start) / (index/len(df)) )/60,2)
            
        
        print("Current progress:", np.round(index/len(df) * 100, 2), "%")
        print("Current run time: ", np.round((stop-start)/60,2), "minutes")
        print("Expected run time", expected_time, "minutes")
    df[name] = county
    return df
        
df = checkPointSigns(df, signs, "Signs")
df

Current progress: 13.0 %
Current run time:  6.04 minutes
Expected run time 46.46 minutes
